In [1]:
from tempo.serve.metadata import ModelFramework
from tempo.serve.model import Model
from tempo.seldon.docker import SeldonDockerRuntime
from tempo.kfserving.protocol import KFServingV2Protocol
from tempo.serve.utils import pipeline, predictmethod
from tempo.seldon.k8s import SeldonKubernetesRuntime
from tempo.serve.metadata import ModelFramework, KubernetesOptions

import numpy as np
import os 
import pprint
import dill

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from alibi.explainers import AnchorTabular
from alibi.datasets import fetch_adult

adult = fetch_adult()
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map = adult.category_map

In [3]:
np.random.seed(0)
data_perm = np.random.permutation(np.c_[data, target])
data = data_perm[:,:-1]
target = data_perm[:,-1]
idx = 30000
X_train,Y_train = data[:idx,:], target[:idx]
X_test, Y_test = data[idx+1:,:], target[idx+1:]

In [ ]:
ordinal_features = [x for x in range(len(feature_names)) if x not in list(category_map.keys())]
ordinal_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                      ('scaler', StandardScaler())])
categorical_features = list(category_map.keys())
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', ordinal_transformer, ordinal_features),
                                               ('cat', categorical_transformer, categorical_features)])
clf = RandomForestClassifier(n_estimators=50)
model=Pipeline(steps=[("preprocess",preprocessor),("model",clf)])
model.fit(X_train,Y_train)


print('Train accuracy: ', accuracy_score(Y_train, model.predict(X_train)))
print('Test accuracy: ', accuracy_score(Y_test, model.predict(X_test)))

In [ ]:
from alibi.explainers import AnchorTabular
predict_fn = lambda x: model.predict(x)
explainer = AnchorTabular(predict_fn, feature_names, categorical_names=category_map, seed=1)
explainer.fit(X_train, disc_perc=[25, 50, 75])

In [ ]:
explanation = explainer.explain(X_test[0], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

In [ ]:
from joblib import dump
dump(model, os.getcwd()+'/income_model/model.joblib') 
with open(os.getcwd()+"/income_explainer/explainer.dill", 'wb') as f:
    dill.dump(explainer,f)

In [4]:
k8s_options = KubernetesOptions(namespace="production")
k8s_runtime = SeldonKubernetesRuntime(k8s_options=k8s_options)

sklearn_model = Model(
        name="income-sklearn",
        runtime=SeldonDockerRuntime(),
        platform=ModelFramework.SKLearn,
        local_folder=os.getcwd()+"/income_model",
        uri="gs://seldon-models/sklearn/income/model-0.23.2"
)


In [ ]:
sklearn_model.deploy()

In [ ]:
sklearn_model(X_test[0:1])

In [5]:
from alibi.utils.wrappers import ArgmaxTransformer
from typing import Any
import os

@pipeline(name="mypipeline",
          runtime=SeldonDockerRuntime(protocol=KFServingV2Protocol()),
          uri="gs://seldon-models/test/income/explainer",
          local_folder=os.getcwd()+"/income_explainer",
          models=[sklearn_model])
class ExplainerPipeline(object):

    def __init__(self):
        if "MLSERVER_MODELS_DIR" in os.environ:
            models_folder = ""
        else:
            models_folder = "/income_explainer"
        with open(os.getcwd()+models_folder+"/explainer.dill", "rb") as f:
            self.explainer = dill.load(f)
        self.ran_init = True
        
    def update_predict_fn(self, x):
        if np.argmax(sklearn_model(x).shape) == 0:
            self.explainer.predictor = sklearn_model
            self.explainer.samplers[0].predictor = sklearn_model
        else:
            self.explainer.predictor = ArgmaxTransformer(sklearn_model)
            self.explainer.samplers[0].predictor = ArgmaxTransformer(sklearn_model)

    @predictmethod
    def explain(self, payload: np.ndarray) -> str:
        if not self.ran_init:
            print("Loading explainer")
            self.__init__()
        self.update_predict_fn(payload)
        explanation = self.explainer.explain(payload, threshold=0.95)
        return explanation.to_json()

In [6]:
p = ExplainerPipeline() 

In [ ]:
p.pipeline.details

In [ ]:
p.explain(X_test[0:1])

In [ ]:
p.save(save_env=False)

In [ ]:
p.set_runtime(SeldonDockerRuntime(protocol=KFServingV2Protocol()))

In [7]:
p.deploy()

In [8]:
p.remote(payload=X_test[0:1])

'{"meta": {"name": "AnchorTabular", "type": ["blackbox"], "explanations": ["local"], "params": {"seed": 1, "disc_perc": [25, 50, 75], "threshold": 0.95, "delta": 0.1, "tau": 0.15, "batch_size": 100, "coverage_samples": 10000, "beam_size": 1, "stop_on_first": false, "max_anchor_size": null, "min_samples_start": 100, "n_covered_ex": 10, "binary_cache_size": 10000, "cache_margin": 1000, "verbose": false, "verbose_every": 1, "kwargs": {}}}, "data": {"anchor": ["Marital Status = Separated", "Sex = Female"], "precision": 0.9565217391304348, "coverage": 0.10723333333333333, "raw": {"feature": [3, 7], "mean": [0.853454821564161, 0.9565217391304348], "precision": [0.853454821564161, 0.9565217391304348], "coverage": [0.18043333333333333, 0.10723333333333333], "examples": [{"covered_true": [[54, 4, 4, 2, 2, 0, 4, 1, 0, 0, 40, 9], [41, 4, 4, 2, 2, 5, 4, 0, 0, 0, 20, 9], [46, 1, 0, 2, 1, 4, 4, 0, 0, 0, 40, 9], [33, 7, 1, 2, 5, 1, 4, 1, 0, 0, 20, 9], [40, 2, 5, 2, 5, 1, 4, 0, 0, 0, 40, 9], [23, 2, 4

In [ ]:
p.undeploy()

In [ ]:
X_test[0:1].shape

In [ ]:
p.request({"inputs":[{"name":"payload","datatype":"INT64","data":[52,  4,  0,  2,  8,  4,  2,  0,  0,  0, 60,  9],"shape":[1,12]}]})